In [1]:
from dotenv import load_dotenv

load_dotenv()  # This loads the .env file

import os

import importlib

import agent
import prompt

importlib.reload(agent)
importlib.reload(prompt)

from agent import create_agent
from prompt import build_restaurant_prompt


graph = create_agent()

graph.get_graph().draw_mermaid_png(output_file_path="graph.png")

# Thread-aware config: persist messages per user/thread for memory
thread_config = {
    "configurable": {
        "thread_id": os.getenv("THREAD_ID", "demo-thread"),
        "user_id": os.getenv("USER_ID", "demo-user"),
    }
}

# Build request from prompt template (flexible by location)
request = build_restaurant_prompt(location="Shinjuku, Tokyo, Japan")

INFO:agent:Creating agent
INFO:agent:Memory created
INFO:agent:Graph created


In [2]:
# Stream with cleaner output while capturing inner state
print("=== Streaming Agent Output ===")
final_state = None
for event in graph.stream(request, config=thread_config):
    if "model" in event:
        msg = event["model"]["messages"]
        try:
            content = getattr(msg, "content", None) or str(msg)
        except Exception:
            content = str(msg)
        if content:
            print(content)
    final_state = event

print("\n=== Final Inner State Snapshot ===")
print(final_state)

=== Streaming Agent Output ===


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:tool:GOOGLE_MAPS_API_KEY is set
INFO:tool:session is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:photo_ref is set
INFO:tool:Results are set
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



=== Final Inner State Snapshot ===
{'agent': {'messages': [AIMessage(content='Here are the top restaurants around Shinjuku, Tokyo, Japan that specialize in Japanese cuisine:\n\n1. **Barubora-ya**\n   - **Rating:** 4.5\n   - **Reviews Count:** 97\n   - **Price Range:** Not specified\n   - **Types:** Restaurant, Food, Point of Interest, Establishment\n   - **Photo URL:** ![Barubora-ya](https://maps.googleapis.com/maps/api/place/photo?maxwidth=800&photo_reference=AciIO2cRtvPicGFgxsxPLFGcNH-UM7wNxXm80e2UAMEczvEgGAcXVG9tcvX2tHDsGqwaJmyO8_iv-3kbLZrPUXOVDyGVayR92-Q9UaScbBbMFuwy3EqJJNXZZToqG3FovmvwwuO05CQHsRnBO71OhxIcTeabGRMpXEVnHE0L_Gf4dQP4uOxYuxEwwijKNTvG5HmtUl7UGhxAzxDGByyzEbxs6LUqQfIY-GEx7SU9xcO_VwQmMnvRufRtoRP5XuiyeH2OCncH-LjNoW8JUPqTVnm_fUIbhUxthgeCNrpZExt65eMxRwRerS1_N_5sY__TYNLtGqcXRobBvxjfzfkHoUagJ3cMumtihwhKQ05c4-i1lpeZRWKhnS2HOC_Yobf5wz0duLs-jcT7pKNklWRXdRocmFLliZYDg9o7g8Qdqc1sbdQ6kceaUhkLcrO7ZBbCMhrPjAdKlIZrm33Z5_kU5i7RJvnMGzrpLjUwhXy1ic7NeoXtGAQiq8E4BiX3yL3XtMF16z_SEm_JuVa3tZ7Q8x

In [3]:
print(final_state["agent"]["messages"][0].content)

Here are the top restaurants around Shinjuku, Tokyo, Japan that specialize in Japanese cuisine:

1. **Barubora-ya**
   - **Rating:** 4.5
   - **Reviews Count:** 97
   - **Price Range:** Not specified
   - **Types:** Restaurant, Food, Point of Interest, Establishment
   - **Photo URL:** ![Barubora-ya](https://maps.googleapis.com/maps/api/place/photo?maxwidth=800&photo_reference=AciIO2cRtvPicGFgxsxPLFGcNH-UM7wNxXm80e2UAMEczvEgGAcXVG9tcvX2tHDsGqwaJmyO8_iv-3kbLZrPUXOVDyGVayR92-Q9UaScbBbMFuwy3EqJJNXZZToqG3FovmvwwuO05CQHsRnBO71OhxIcTeabGRMpXEVnHE0L_Gf4dQP4uOxYuxEwwijKNTvG5HmtUl7UGhxAzxDGByyzEbxs6LUqQfIY-GEx7SU9xcO_VwQmMnvRufRtoRP5XuiyeH2OCncH-LjNoW8JUPqTVnm_fUIbhUxthgeCNrpZExt65eMxRwRerS1_N_5sY__TYNLtGqcXRobBvxjfzfkHoUagJ3cMumtihwhKQ05c4-i1lpeZRWKhnS2HOC_Yobf5wz0duLs-jcT7pKNklWRXdRocmFLliZYDg9o7g8Qdqc1sbdQ6kceaUhkLcrO7ZBbCMhrPjAdKlIZrm33Z5_kU5i7RJvnMGzrpLjUwhXy1ic7NeoXtGAQiq8E4BiX3yL3XtMF16z_SEm_JuVa3tZ7Q8xh1ox2yhTFcIHKHSNK-wYhSq4bNLGkVmDtqOBdWI55oe5g0CFdLtZoV9CZKRQ&key=AIzaSyB-GCxF8vWXNN-9g